In [172]:
import codecs

In [17]:
CHARS_IN_GRAM = 3

In [295]:
# globals, set by create_index()
SEARCH_INDEX        = dict()
SEARCH_INDEX_LENGTH = 0

In [296]:
def create_index(data):
    global SEARCH_INDEX, SEARCH_INDEX_LENGTH
    SEARCH_INDEX = dict()
    for i in range(0, len(data) - CHARS_IN_GRAM + 1):
        gram = data[i:i+CHARS_IN_GRAM]
        if not gram in SEARCH_INDEX:
            SEARCH_INDEX[gram] = list()
        SEARCH_INDEX[gram].append(i)
    SEARCH_INDEX_LENGTH = i

In [198]:
def search_score_naive(term):
    """
    Score basiert lediglich auf der Anzahl der Vorkommen der n-grams im Suchindex.
    """
    term_index = create_index(term)
    matches = 0
    term_grams = [term[i:i+CHARS_IN_GRAM] for i in range(0, len(term) - CHARS_IN_GRAM + 1)]
    for gram in term_grams:
        if gram in SEARCH_INDEX:
            position = search_index[gram]
            matches += 1
    return matches / len(term_grams)

In [250]:
sentence = "Пеппи сидела на диване и молча слушала разговор дам"

In [251]:
create_index(sentence)

In [252]:
search_score_naive("слушал")

1.0

In [310]:
search_score_naive("Пппи диване")

KeyError: 'Ппп'

In [254]:
search_score_naive("молча слушала")

1.0

In [255]:
# Zu großer Score
search_score_naive("молча разговор")

1.0

In [256]:
def create_gram_list(data):
    return [data[i:i+CHARS_IN_GRAM] for i in range(0, len(data) - CHARS_IN_GRAM + 1)]

In [297]:
def search_score(term):
    """
    Score basiert auf Abständen der n-grams zum ersten n-gram, der ein Treffer ist.
    """
    score = 0
    position_first_match = None
    term_grams = create_gram_list(term)
    if len(term_grams) == 0:
        # Avoid division by zero
        return 0.0
    for position_in_term, gram in enumerate(term_grams):
        # Initialise: not in index
        positions = [float("inf")]
        if gram not in SEARCH_INDEX:
            continue
            
        positions = SEARCH_INDEX[gram]
        
        if not position_first_match:
            # TODO: this is a simplification. Takes always first match
            position_first_match = positions[0]
        
        # Determine best match and it's distance
        min_dist = float("inf")
        for position in positions:
            dist = abs(position - position_first_match - position_in_term)
            if dist < min_dist: min_dist = dist
        
        # Avoid division by zero
        if min_dist == 0: min_dist = 1
        
        
        score_gram = 1 - min_dist / SEARCH_INDEX_LENGTH
        print("{}, Position {}, rel. distance {}, score gram {}".format(gram, position, min_dist, score_gram))
        score += score_gram
    return score/len(term_grams)

In [298]:
create_index(sentence)

In [299]:
search_score("слушала")

слу, Position 31, rel. distance 1, score gram 0.9791666666666666
луш, Position 32, rel. distance 1, score gram 0.9791666666666666
уша, Position 33, rel. distance 1, score gram 0.9791666666666666
шал, Position 34, rel. distance 1, score gram 0.9791666666666666
ала, Position 35, rel. distance 1, score gram 0.9791666666666666


0.9791666666666666

In [300]:
search_score("слушали")

слу, Position 31, rel. distance 1, score gram 0.9791666666666666
луш, Position 32, rel. distance 1, score gram 0.9791666666666666
уша, Position 33, rel. distance 1, score gram 0.9791666666666666
шал, Position 34, rel. distance 1, score gram 0.9791666666666666


0.7833333333333333

In [301]:
search_score("молча слушала")

мол, Position 25, rel. distance 1, score gram 0.9791666666666666
олч, Position 26, rel. distance 1, score gram 0.9791666666666666
лча, Position 27, rel. distance 1, score gram 0.9791666666666666
ча , Position 28, rel. distance 1, score gram 0.9791666666666666
а с, Position 29, rel. distance 1, score gram 0.9791666666666666
 сл, Position 30, rel. distance 1, score gram 0.9791666666666666
слу, Position 31, rel. distance 1, score gram 0.9791666666666666
луш, Position 32, rel. distance 1, score gram 0.9791666666666666
уша, Position 33, rel. distance 1, score gram 0.9791666666666666
шал, Position 34, rel. distance 1, score gram 0.9791666666666666
ала, Position 35, rel. distance 1, score gram 0.9791666666666666


0.9791666666666665

In [302]:
search_score("слушала молча")

слу, Position 31, rel. distance 1, score gram 0.9791666666666666
луш, Position 32, rel. distance 1, score gram 0.9791666666666666
уша, Position 33, rel. distance 1, score gram 0.9791666666666666
шал, Position 34, rel. distance 1, score gram 0.9791666666666666
ала, Position 35, rel. distance 1, score gram 0.9791666666666666
ла , Position 36, rel. distance 1, score gram 0.9791666666666666
 мо, Position 24, rel. distance 14, score gram 0.7083333333333333
мол, Position 25, rel. distance 14, score gram 0.7083333333333333
олч, Position 26, rel. distance 14, score gram 0.7083333333333333
лча, Position 27, rel. distance 14, score gram 0.7083333333333333


0.7916666666666665

In [303]:
search_score("Пеппи дам диван")

Пеп, Position 0, rel. distance 1, score gram 0.9791666666666666
епп, Position 1, rel. distance 1, score gram 0.9791666666666666
ппи, Position 2, rel. distance 1, score gram 0.9791666666666666
пи , Position 3, rel. distance 1, score gram 0.9791666666666666
 да, Position 47, rel. distance 41, score gram 0.14583333333333337
дам, Position 48, rel. distance 41, score gram 0.14583333333333337
 ди, Position 15, rel. distance 5, score gram 0.8958333333333334
див, Position 16, rel. distance 5, score gram 0.8958333333333334
ива, Position 17, rel. distance 5, score gram 0.8958333333333334
ван, Position 18, rel. distance 5, score gram 0.8958333333333334


0.5993589743589742

In [304]:
search_score("Пеппи диван")

Пеп, Position 0, rel. distance 1, score gram 0.9791666666666666
епп, Position 1, rel. distance 1, score gram 0.9791666666666666
ппи, Position 2, rel. distance 1, score gram 0.9791666666666666
пи , Position 3, rel. distance 1, score gram 0.9791666666666666
 ди, Position 15, rel. distance 9, score gram 0.8125
див, Position 16, rel. distance 9, score gram 0.8125
ива, Position 17, rel. distance 9, score gram 0.8125
ван, Position 18, rel. distance 9, score gram 0.8125


0.7962962962962963

In [305]:
search_score(sentence)

Пеп, Position 0, rel. distance 1, score gram 0.9791666666666666
епп, Position 1, rel. distance 1, score gram 0.9791666666666666
ппи, Position 2, rel. distance 1, score gram 0.9791666666666666
пи , Position 3, rel. distance 1, score gram 0.9791666666666666
и с, Position 4, rel. distance 1, score gram 0.9791666666666666
 си, Position 5, rel. distance 1, score gram 0.9791666666666666
сид, Position 6, rel. distance 1, score gram 0.9791666666666666
иде, Position 7, rel. distance 1, score gram 0.9791666666666666
дел, Position 8, rel. distance 1, score gram 0.9791666666666666
ела, Position 9, rel. distance 1, score gram 0.9791666666666666
ла , Position 36, rel. distance 1, score gram 0.9791666666666666
а н, Position 11, rel. distance 1, score gram 0.9791666666666666
 на, Position 12, rel. distance 1, score gram 0.9791666666666666
на , Position 13, rel. distance 1, score gram 0.9791666666666666
а д, Position 14, rel. distance 1, score gram 0.9791666666666666
 ди, Position 15, rel. distance 1, 

0.9791666666666662

In [306]:
document1 = ""
with codecs.open("draka.txt", "r", "utf-8") as h:
    document1 = h.read()
create_index(document1)

In [307]:
search_score("побежали")

поб, Position 411, rel. distance 1, score gram 0.9999228692634015
обе, Position 12592, rel. distance 1, score gram 0.9999228692634015
беж, Position 6954, rel. distance 1, score gram 0.9999228692634015
ежа, Position 6955, rel. distance 1, score gram 0.9999228692634015
жал, Position 11586, rel. distance 1, score gram 0.9999228692634015
али, Position 8664, rel. distance 1, score gram 0.9999228692634015


0.9999228692634015

In [308]:
search_score("побежали в XXX ванную, XXX")

поб, Position 411, rel. distance 1, score gram 0.9999228692634015
обе, Position 12592, rel. distance 1, score gram 0.9999228692634015
беж, Position 6954, rel. distance 1, score gram 0.9999228692634015
ежа, Position 6955, rel. distance 1, score gram 0.9999228692634015
жал, Position 11586, rel. distance 1, score gram 0.9999228692634015
али, Position 8664, rel. distance 1, score gram 0.9999228692634015
ли , Position 12705, rel. distance 1, score gram 0.9999228692634015
и в, Position 12835, rel. distance 1, score gram 0.9999228692634015
 в , Position 12836, rel. distance 1, score gram 0.9999228692634015
 ва, Position 10064, rel. distance 4, score gram 0.9996914770536058
ван, Position 5051, rel. distance 4, score gram 0.9996914770536058
анн, Position 12463, rel. distance 4, score gram 0.9996914770536058
нну, Position 5833, rel. distance 4, score gram 0.9996914770536058
ную, Position 12504, rel. distance 4, score gram 0.9996914770536058
ую,, Position 12352, rel. distance 4, score gram 0.9996

0.6665477567810772